# MGTA 415 - Session 0 Demo: A simple example of paper trading with Alpaca API

This notebook demonstrates how to use the Alpaca API to pull real-time news, analyze sentiment, and make trading decisions based on the sentiment analysis.

## Step 1: Import Libraries

First, we'll import the necessary libraries and import the secrets. 
This assumes the secrets are already added in .env file.  Everytime you update the .env file you'll need to rebuild the container.

In [1]:
import os
import alpaca_trade_api as tradeapi
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import requests
from dotenv import load_dotenv
import os

In [2]:
ALPACA_URL = 'https://paper-api.alpaca.markets'
SYMBOLS = ["AAPL"]

In [3]:
# Alpaca API credentials
load_dotenv()

ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

# Initialize Alpaca API
alpaca = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url=ALPACA_URL)

# Initialize Sentiment Analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

## Step 2: Retrieve News from Previous Day

We'll retrieve news articles from the previous day using Alpaca's API.


In [4]:
# Function to retrieve news from the previous day for specific symbols
def get_previous_day_news(symbols):
    today = datetime.utcnow()
    yesterday = today - timedelta(days=1)
    url = "https://data.alpaca.markets/v1beta1/news"
    headers = {
        "APCA-API-KEY-ID": ALPACA_API_KEY,
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY
    }
    params = {
        "symbols": ",".join(symbols),
        "start": yesterday.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "end": today.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "limit": 50
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()['news']

# Retrieve news for specific symbols
news = get_previous_day_news(SYMBOLS)


HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v1beta1/news?symbols=AAPL&start=2024-08-30T21%3A06%3A42Z&end=2024-08-31T21%3A06%3A42Z&limit=50

In [10]:
news

[{'author': 'Mark Putrino',
  'content': '',
  'created_at': '2024-08-13T19:33:24Z',
  'headline': 'Stock Of The Day: Is Apple The Alpha Stock?',
  'id': 40349969,
  'images': [{'size': 'large',
    'url': 'https://cdn.benzinga.com/files/imagecache/2048x1536xUP/images/story/2024/08/13/New-York--Ny--Usa---July-9--2022-Apple-L.jpeg'},
   {'size': 'small',
    'url': 'https://cdn.benzinga.com/files/imagecache/1024x768xUP/images/story/2024/08/13/New-York--Ny--Usa---July-9--2022-Apple-L.jpeg'},
   {'size': 'thumb',
    'url': 'https://cdn.benzinga.com/files/imagecache/250x187xUP/images/story/2024/08/13/New-York--Ny--Usa---July-9--2022-Apple-L.jpeg'}],
  'source': 'benzinga',
  'summary': 'Shares of Apple may be breaking out. AAPL is the biggest stock in the S&amp;P 500. These dynamics could influence the entire market.',
  'symbols': ['AAPL'],
  'updated_at': '2024-08-13T20:32:38Z',
  'url': 'https://www.benzinga.com/trading-ideas/long-ideas/24/08/40349969/stock-of-the-day-is-apple-the-alph

## Step 3: Analyze Sentiment and Make Trading Decisions

We'll analyze the sentiment of the news articles and make trading decisions based on the sentiment scores.


In [8]:
# Function to analyze sentiment and make trading decisions
def analyze_and_trade(news):
    for article in news:
        headline = article['headline']
        summary = article.get('summary', '')
        content = article.get('content', '')
        combined_text = f"{headline} {summary} {content}"
        
        sentiment = sentiment_analyzer.polarity_scores(combined_text)
        sentiment_score = sentiment['compound']
        
        print(f"Headline: {headline}")
        print(f"Sentiment Score: {sentiment_score}")
        
        symbols = article['symbols']
        for symbol in symbols:
            if sentiment_score > 0.05:
                make_trade_decision(symbol, 'buy')
            elif sentiment_score < -0.05:
                make_trade_decision(symbol, 'sell')

In [9]:
# Function to make trading decisions
def make_trade_decision(symbol, action):
    qty = 10  # Example quantity
    
    try:
        if action == 'buy':
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
            print(f"Buying {qty} shares of {symbol}")
        elif action == 'sell':
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
            print(f"Selling {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")


# Analyze and trade based on the news
analyze_and_trade(news)

Headline: Stock Of The Day: Is Apple The Alpha Stock?
Sentiment Score: 0.5106
Buying 10 shares of AAPL
Headline: Apple Shares Are Up Today: What You Need To Know
Sentiment Score: 0.5267
Buying 10 shares of AAPL
Headline: Tech Sector Volatility Presents Opportunities With Direxion's Leveraged ETFs
Sentiment Score: 0.3818
Buying 10 shares of AAPL
Buying 10 shares of AVGO
Buying 10 shares of MSFT
Buying 10 shares of NVDA
Buying 10 shares of TECL
Buying 10 shares of TECS
Headline: Apple, Intel, And AMD Lead Charge In AI-Capable PC Market Surge: Report
Sentiment Score: 0.1779
Buying 10 shares of AAPL
Buying 10 shares of AMD
Buying 10 shares of DELL
Buying 10 shares of HPQ
Buying 10 shares of INTC
Buying 10 shares of QCOM
Headline: 10 Information Technology Stocks Whale Activity In Today's Session
Sentiment Score: 0.0
Headline: Pay Attention: Inflation Headline Is Misleading, Home Depot Earnings Show Consumer Weakness
Sentiment Score: -0.3612
Error placing order: potential wash trade detecte